In [14]:
import ee
ee.Authenticate()


Successfully saved authorization token.


In [21]:
import geemap

DEM = ee.Image("USGS/SRTMGL1_003") # elevation map

#node1 = [-72, -42]
#node2 = [-73, -42]
#node3 = [-72, -43]
#node4 = [103, 3]

#nodes = [node1, node2, node3, node4]

import pandas as pd
df = pd.read_csv('regression_data.csv')
nodes = []

for i in range(8000, 12000):
    temp = [df['longitude'][i], df['latitude'][i]]
    nodes.append(temp)

# make points from nodes
points = [ee.Geometry.Point(coord) for coord in nodes]

# make features from points (name by list order)
feats = [ee.Feature(p, {'name': 'node{}'.format(i)}) for i, p in enumerate(points)]

# make a featurecollection from points
fc = ee.FeatureCollection(feats)

# extract points from DEM
reducer = ee.Reducer.first()
data = DEM.reduceRegions(fc, reducer.setOutputs(['elevation']), 30)

geemap.ee_to_csv(data, filename='altitude3.csv')

# see data
#for feat in data.getInfo()['features']:
    # print(feat['properties'])

# export as CSV
# export as CSV
# task = ee.batch.Export.table.toDrive(data, 'pointsDataExtract', 'FOLDER', 'nameOfCSV')
#task = ee.batch.Export.table.toDrive(collection=data, folder='data_earth_engine', description='altitude', fileFormat='CSV')
#ee.batch.Task(task.id, "EXPORT_FEATURES", "UNSUBMITTED", task.config).start()

Generating URL ...
Please wait ...
Data downloaded to c:\Users\Khor Kean Teng\Downloads\Flood Risk Model 2\altitude3.csv


In [12]:
import pandas as pd
import ee
import geemap

ee.Initialize()

data = pd.DataFrame({
    'longitude': [102.13],
    'latitude': [3.123],
})

def get_elevation(data):
    DEM = ee.Image("USGS/SRTMGL1_003")
    
    nodes = [data['longitude'][0], data['latitude'][0]]
    points = [ee.Geometry.Point(nodes)]
    feats = [ee.Feature(p, {'name': 'node{}'.format(i)}) for i, p in enumerate(points)]
    fc = ee.FeatureCollection(feats)
    reducer = ee.Reducer.first()
    data = DEM.reduceRegions(fc, reducer.setOutputs(['elevation']), 30)
    elevation = data.getInfo()['features'][0]['properties']['elevation']
    return elevation

x = get_elevation(data)
print(x)

418


For ee, we can only make maximum of 5000 requests per execution. So I split the process into 4000 rows per batch